In [1]:
import cv2
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [2]:
# Function to preprocess video files into frames
def preprocess_video(video_path, num_frames=30, frame_height=128, frame_width=128):
    cap = cv2.VideoCapture(video_path)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frames = []
    for _ in range(num_frames):
        frame_index = np.random.randint(0, frame_count)
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, frame = cap.read()
        if ret:
            frame = cv2.resize(frame, (frame_height, frame_width))
            frames.append(frame)
    cap.release()
    return frames

In [3]:
# Load and preprocess all videos in your dataset
X = []
y = []

In [4]:
fake_video_dir = r'C:\Users\Master\Documents\Deepfake Detection\fake'
real_video_dir = r'C:\Users\Master\Documents\Deepfake Detection\real'



In [5]:
# Process fake videos
for video_file in os.listdir(fake_video_dir):
    video_path = os.path.join(fake_video_dir, video_file)
    frames = preprocess_video(video_path)
    X.append(frames)
    y.append(1)  # Label 1 for fake videos


In [6]:
# Process real videos
for video_file in os.listdir(real_video_dir):
    video_path = os.path.join(real_video_dir, video_file)
    frames = preprocess_video(video_path)
    X.append(frames)
    y.append(0)  # Label 0 for real videos


In [7]:
# Convert lists to numpy arrays
X = np.array(X)
y = np.array(y)

In [8]:
# Split the data into train, validation, and test sets
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)


In [13]:
from keras.models import Sequential
from keras.layers import Conv3D, MaxPooling3D, Flatten, Dense, Dropout
model = Sequential([
    Conv3D(32, (3, 3, 3), activation='relu', input_shape=(30, 128, 128, 3)),
    MaxPooling3D((2, 2, 2)),
    Conv3D(64, (3, 3, 3), activation='relu'),
    MaxPooling3D((2, 2, 2)),
    Conv3D(128, (3, 3, 3), activation='relu'),
    MaxPooling3D((2, 2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [14]:
# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])



In [16]:
# Train your CNN (assuming you have already defined your model)
NUM_EPOCHS = 10
history = model.fit(X_train, y_train, epochs=NUM_EPOCHS, validation_data=(X_val, y_val))


Epoch 1/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 132s 12s/step - accuracy: 0.4285 - loss: 107.4855 - val_accuracy: 0.4250 - val_loss: 0.7241
Epoch 2/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 93s 8s/step - accuracy: 0.4786 - loss: 0.7261 - val_accuracy: 0.4750 - val_loss: 0.6882
Epoch 3/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 63s 6s/step - accuracy: 0.5436 - loss: 0.6773 - val_accuracy: 0.4750 - val_loss: 0.6786
Epoch 4/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 76s 8s/step - accuracy: 0.6135 - loss: 0.6934 - val_accuracy: 0.4000 - val_loss: 0.6997
Epoch 5/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 64s 6s/step - accuracy: 0.6605 - loss: 0.6326 - val_accuracy: 0.3750 - val_loss: 0.7148
Epoch 6/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 62s 6s/step - accuracy: 0.6666 - loss: 0.6148 - val_accuracy: 0.4500 - val_loss: 0.7081
Epoch 7/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 65s 6s/step - accuracy: 0.6827 - loss: 0.5644 - val_accuracy: 0.3750 - val_loss: 0.7229
Epoch 8/10
10/10 ━━━━━━━━━━━━━━━━━━━━ 48s 5s/step - accuracy: 0.6776 - loss: 0.5582 - val_accuracy: 0.4500 - val_l

In [18]:
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test,y_test)


2/2 ━━━━━━━━━━━━━━━━━━━━ 3s 261ms/step - accuracy: 0.4354 - loss: 0.9374


In [19]:
# Save the model
model.save('deepfake_detection_model.h5')

### Predicting a video is fake or real

In [90]:
# Input a test video and preprocess
test_frames=[]
video_frames = preprocess_video('testv1.mp4')
test_frames.append(video_frames)
test_frames = np.array(test_frames)
Deepfakedetection = load_model('deepfake_detection_model.h5')
predictions=Deepfakedetection.predict(test_frames)
# Funcion for post processing the predictions
def postprocess_predictions(predictions):
    processed_predictions = (predictions > 0.5).astype(int)
    return processed_predictions
# Final predictions
processed_predictions = postprocess_predictions(predictions)
if processed_predictions==1:
    print("The video is Real")
else:
    print("The video is Fake")  

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 190ms/step
The video is Fake


In [92]:
import tkinter as tk
from tkinter import filedialog
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
from PIL import Image, ImageTk

In [95]:
def find_deepfake():
    video_path = filedialog.askopenfilename()
    test_frames=[]
    video_frames = preprocess_video(video_path)
    test_frames.append(video_frames)
    test_frames = np.array(test_frames)
    Deepfakedetection = load_model('deepfake_detection_model.h5')
    predictions=Deepfakedetection.predict(test_frames)
    processed_predictions = (predictions > 0.5).astype(int)
    result_label.config(text="The video is Real" if np.all(processed_predictions == 1) else "The video is Fake")

In [93]:
# Define the Tkinter GUI


''

In [96]:
root = tk.Tk()
root.title("Deep Fake Detection")
# Create a label for displaying the result
result_label = tk.Label(root, text="")
result_label.pack(pady=10)

# Create a button to upload the video
upload_button = tk.Button(root, text="Upload Video", command=find_deepfake)
upload_button.pack(pady=10)

# Run the Tkinter event loop
root.mainloop()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 185ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 150ms/step


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 125ms/step


In [98]:
import tkinter as tk
from tkinter import filedialog, ttk
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
from PIL import Image, ImageTk

In [3]:
# Define the Tkinter GUI
root = tk.Tk()
root.title("Deep Fake Detection")
root.geometry("800x600")  # Set the size of the root window

# Define styles
style = ttk.Style()
style.configure('TButton', padding=10, font=('Helvetica', 14))
style.configure('TLabel', padding=10, font=('Helvetica', 14))
style.configure('TFrame', padding=10)

# Create a frame for the header
header_frame = ttk.Frame(root)
header_frame.pack(pady=20)

# Create a label for the header
header_label = ttk.Label(header_frame, text="Deep Fake Detection", style='TLabel')
header_label.pack()

# Create a frame for the video upload section
upload_frame = ttk.Frame(root)
upload_frame.pack(pady=20)

# Create a button to upload the video
upload_button = ttk.Button(upload_frame, text="Upload Video", style='TButton', command=find_deepfake)
upload_button.pack(side='left', padx=10)

# Create a label to display the result
result_label = ttk.Label(upload_frame, text="", style='TLabel')
result_label.pack(side='left')

# Run the Tkinter event loop
root.mainloop()

NameError: name 'find_deepfake' is not defined

In [2]:
import tkinter as tk
from tkinter import filedialog, ttk
import cv2
import numpy as np
from keras.models import load_model
from keras.preprocessing import image
from PIL import Image, ImageTk

# Function to preprocess the video frames
def preprocess_video(video_path):
    frames = []
    cap = cv2.VideoCapture(video_path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = cv2.resize(frame, (128, 128))  # Resize the frame
        frames.append(frame)
    cap.release()
    return np.array(frames)

# Function to check if the video is real or fake
def check_video():
    global video_frames
    
    # Make predictions using the model
    Deepfakedetection = load_model('deepfake_detection_model.h5')
    predictions=Deepfakedetection.predict(video_frames)
    
    # Post-process the predictions
    processed_predictions = postprocess_predictions(predictions)
    
    # Display the result
    result_label.config(text="The uploaded video is Real" if np.all(processed_predictions == 1) else "The uploaded video is Fake")

# Function to handle video upload
def upload_video():
    global video_frames
    
    # Get the selected video file
    video_path = filedialog.askopenfilename()
   
    # Preprocess the video frames
    video_frames = preprocess_video(video_path)
    
    # Display the first frame of the video
    if len(video_frames) > 0:
        frame_image = Image.fromarray(video_frames[0])
        frame_image.thumbnail((300, 300))  # Resize the image
        frame_photo = ImageTk.PhotoImage(image=frame_image)
        video_label.config(image=frame_photo)
        video_label.image = frame_photo



# Define the Tkinter GUI
root = tk.Tk()
root.title("Deep Fake Detection")
root.geometry("800x600")  # Set the size of the root window
root.configure(bg="green") 

# Define styles
style = ttk.Style()
style.configure('TButton', padding=10, font=('Helvetica', 14),background='lightgray')
style.configure('TLabel', padding=10, font=('Times New Roman', 25,'bold'),background='green')
style.configure('TFrame', padding=10,background='green')

# Create a frame for the header
header_frame = ttk.Frame(root)
header_frame.pack(pady=20)

# Create a label for the header
header_label = ttk.Label(header_frame, text="DEEPFAKE CATCHER", style='TLabel',foreground='white')
header_label.pack()

# Create a frame for the video display
video_frame = ttk.Frame(root)
video_frame.pack(pady=20)

# Create a label to display the uploaded video
video_label = ttk.Label(video_frame, style='TLabel')
video_label.pack()

# Create a frame for the video upload section
upload_frame = ttk.Frame(root)
upload_frame.pack(pady=20)

# Create a button to upload the video
upload_button = ttk.Button(upload_frame, text="Upload Video", style='TButton', command=upload_video)
upload_button.pack(side='left', padx=20)

# Create a button to check the video
check_button = ttk.Button(upload_frame, text="Check Video", style='TButton', command=check_video)
check_button.pack(side='left', padx=20)


# Create a frame for the result label
result_frame = ttk.Frame(root)
result_frame.pack(pady=20)

# Create a label to display the result
result_label = ttk.Label(result_frame, text="", style='TLabel',foreground='lightgray')
result_label.pack()

# Initialize variable to store video frames
video_frames = []

# Run the Tkinter event loop
root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\Master\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Master\AppData\Local\Temp\ipykernel_14976\1134888224.py", line 29, in check_video
    predictions=Deepfakedetection.predict(video_frames)
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Master\anaconda3\Lib\site-packages\keras\src\utils\traceback_utils.py", line 122, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "C:\Users\Master\anaconda3\Lib\site-packages\keras\src\models\functional.py", line 280, in _adjust_input_rank
    raise ValueError(
ValueError: Exception encountered when calling Sequential.call().

Invalid input shape for input Tensor("sequential_1_1/Cast:0", shape=(32, 128, 128, 3), dtype=float32). Expected shape (None, 30, 128, 128, 3), but input has incompatible shape (32, 128, 128, 3)

Arguments received

In [109]:
video_frames[0].shape

(30, 128, 128, 3)